In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import edgar_datafied
import sec
from Humble_Chuck import mysql_helper1 as sql
from urllib.request import urlopen
from urllib.request import urlretrieve
from zipfile import ZipFile
import config
import os
import sqlalchemy

_sql_alchemy_connection = (
                                f'mysql+mysqlconnector://'
                                f'{config.admin}:{config.password}'
                                f'@{config.host}:{config.port}'
                                f'/{config.schema}'
                           )
db = sqlalchemy.create_engine(_sql_alchemy_connection,
                              echo = False,
                              connect_args = {'ssl_disabled' : True})